In [1]:
import cobra as cb
import pandas as pd

# Load data

In [2]:
rxn_bounds = pd.read_excel('fba_result.xls','ModelReactions')
rxn_bounds.head(2)

,id,direction,compartment,gpr,name,pathway,equation,definition,flux,min_flux,lowerbound,max_flux,upperbound
0,PPA_c0,>,Cytosol_0,CLO1313_RS04240_CDS_1,CustomReaction_c0,NaN,(2) cpd00009[c0] + (2) cpd00067[e0] -> (1) cpd...,(2) Phosphate_c0[c0] + (2) H+_e0[e0] -> (1) H2...,0.000000,0.00000,0,38.7169,1000
1,ACLS_c0,<,Cytosol_0,((CLO1313_RS00530_CDS_1 and CLO1313_RS00535_CD...,pyruvate:pyruvate acetaldehydetransferase (dec...,NaN,(1) cpd00011[c0] + (1) cpd00668[c0] <- (2) cpd...,(1) CO2_c0[c0] + (1) ALCTT_c0[c0] <- (2) Pyruv...,-0.394004,-7.65257,-1000,0.0000,0


In [3]:
ex_bounds = pd.read_excel('fba_result.xls','ModelCompounds')
ex_bounds.head(2)

,id,name,formula,charge,compartment,uptake,min_uptake,lowerbound,max_uptake,upperbound
0,cpd00067_e0,H+_e0,H,1,NaN,-30.6327,-70.3818,-1000,8.89272,1000
1,cpd02701_c0,S-Adenosyl-4-methylthio-2-oxobutanoate_c0,C15H19N5O6S,0,NaN,0.0000,0.0000,-1000,0.00000,0


# Update bounds of normal and exchange reactions

In [4]:
#load model
model = cb.io.read_sbml_model('iCBI676.xml')

# non-exchange reactions
for idx, row in rxn_bounds.iterrows():
    good_id = row['id'].replace('-','_')
    rxn = model.reactions.get_by_id(good_id)
    if (row['lowerbound'] == -1000) and (row['upperbound'] == 0): # The reaction is irreversible but described in the oposite direction
        rxn.lower_bound = row['upperbound']
        rxn.upper_bound = -row['lowerbound']
    else:
        rxn.lower_bound = row['lowerbound']
        rxn.upper_bound = row['upperbound']

#exchanges
for idx, row in ex_bounds.iterrows():
    rxn_id = 'EX_{}'.format(row['id'])
   # if row['name'] is 'Biomass_c0': # Ignore biomass exchange
    #    pass
    if rxn_id in model.reactions:
        rxn = model.reactions.get_by_id(rxn_id)
        rxn.lower_bound = -float(row['upperbound'])
        rxn.upper_bound =  -float(row['lowerbound'])
    else:
        print('Reaction {} not present in the model'.format(rxn_id))

# Update biomass objective function

In [5]:
model.objective = 'BIOMASS_CELLOBIOSE_c0'

# Check that growth is now predicted correctly and save model

In [6]:
model.optimize()
model.summary()

IN FLUXES             OUT FLUXES            OBJECTIVES
--------------------  --------------------  ----------------------
cpd00001_e0  4.93     cpd11640_e0  18.4     BIOMASS_CELL...  0.674
cpd00158_e0  4        cpd00067_e0  13.8
cpd00073_e0  2.28     cpd00011_e0  12.9
cpd00205_e0  0.641    cpd00029_e0   8.06
cpd00009_e0  0.536    cpd00069_e0   0.0182
cpd00254_e0  0.0923
cpd00048_e0  0.0871
cpd10516_e0  0.00313
cpd00063_e0  0.00291


In [7]:
cb.io.write_sbml_model(model,'iCBI665_v2.sbml')